In [13]:
import pandas as pd

# 1. 讀取資料，選擇需要的欄位
df = pd.read_excel("dataMovie.xlsx")
df = df.loc[:, ['movie_title', 'movie_description', 'movie_genre']]
df.head(3)

,movie_title,movie_description,movie_genre
0,刺激1995,銀行家安迪本欲報復外遇的妻子，但在行動之前打消主意，卻不料其妻及其情夫當晚即遭人殺害，安迪遂...,"劇情, 犯罪"
1,教父,40年代的美國，“教父”維托·唐·柯里昂是黑手黨柯里昂家族的首領，帶領家族從事非法的勾當，但...,"劇情, 犯罪"
2,教父II,影片主要講述第二代教父麥克·柯里昂的奮鬥歷程，同時回憶了第一代教父維多·柯里昂創業的艱辛，反...,"劇情, 犯罪"


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9935 entries, 0 to 9934
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movie_title        9935 non-null   object
 1   movie_description  7234 non-null   object
 2   movie_genre        9932 non-null   object
dtypes: object(3)
memory usage: 233.0+ KB


In [15]:
df = df.dropna()

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7234 entries, 0 to 9933
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movie_title        7234 non-null   object
 1   movie_description  7234 non-null   object
 2   movie_genre        7234 non-null   object
dtypes: object(3)
memory usage: 226.1+ KB


In [17]:
import pandas as pd

# 讀取 Excel
df = pd.read_excel("dataMovie.xlsx")

# 移除空值 NaN
df = df.dropna()

# 另存新檔
df.to_excel("dataMovie_clean.xlsx", index=False)

print("已將清理後的資料另存為 dataMovie_clean.xlsx")

已將清理後的資料另存為 dataMovie_clean.xlsx


In [24]:
import pandas as pd
from fuzzywuzzy import process

# 讀取 Excel 檔案
df = pd.read_excel("dataMovie_clean.xlsx")

# 用戶輸入的關鍵字
user_input = "航海王"

# 取得最相似的電影名稱
best_match = process.extractOne(user_input, df["movie_title"])

print(f"最佳匹配: {best_match[0]} (相似度: {best_match[1]})")


最佳匹配: 航海王：Z (相似度: 90)


In [25]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# 讀取 Excel
df = pd.read_excel("dataMovie_clean.xlsx")

# 假設 "Genre" 欄位是以逗號分隔的類別，例如 "科幻, 冒險"
df["movie_genre"] = df["movie_genre"].astype(str).apply(lambda x: x.split(", "))


# 使用 MultiLabelBinarizer 進行 One-Hot Encoding
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(df["movie_genre"])

# 轉為 DataFrame
genre_df = pd.DataFrame(genre_matrix, columns=mlb.classes_)

# 合併回原始 DataFrame
df = df.join(genre_df)

# 顯示結果
print(df.head(3))


   id movie_title                                  movie_description  \
0   1      刺激1995  銀行家安迪本欲報復外遇的妻子，但在行動之前打消主意，卻不料其妻及其情夫當晚即遭人殺害，安迪遂...   
1   2          教父  40年代的美國，“教父”維托·唐·柯里昂是黑手黨柯里昂家族的首領，帶領家族從事非法的勾當，但...   
2   3        教父II  影片主要講述第二代教父麥克·柯里昂的奮鬥歷程，同時回憶了第一代教父維多·柯里昂創業的艱辛，反...   

          release_date vote_average movie_genre  \
0  1994-09-23 00:00:00        8.709    [劇情, 犯罪]   
1  1972-03-14 00:00:00        8.689    [劇情, 犯罪]   
2  1974-12-20 00:00:00          8.6    [劇情, 犯罪]   

                        poster_path   動作  []  [愛情  ...  科幻,恐怖  科幻,愛情  科幻,懸疑  \
0  /lnu10JjbS3dhv38UCg832hmvQj3.jpg    0   0    0  ...      0      0      0   
1  /y03tzUKvkRCYwJ5NWys4W4bnS9m.jpg    0   0    0  ...      0      0      0   
2  /4J8jUmjgKPggRPJqqdkjWCj2k0D.jpg    0   0    0  ...      0      0      0   

   科幻,戰爭  科幻,電視  科幻,驚悚  西部  電視  音樂  驚悚  
0      0      0      0   0   0   0   0  
1      0      0      0   0   0   0   0  
2      0      0      0   0   0   0   0  

[3 rows x 52 columns]


In [33]:
import jieba
from rank_bm25 import BM25Okapi

# === 2️⃣ 建立 BM25 模型來處理電影類型檢索 ===
genre_corpus = [" ".join(genres) for genres in df["movie_genre"].tolist()]  # 把類型合併成字串
tokenized_genre_corpus = [doc.split() for doc in genre_corpus]  # 分詞 (直接根據空格切割)
bm25_genre = BM25Okapi(tokenized_genre_corpus)  # 建立 BM25 模型


# 用戶輸入
user_query = "冒險, 動作"
tokenized_query = list(jieba.cut(user_query))

# 計算 BM25 分數
scores = bm25_genre.get_scores(tokenized_query)

# 找到最相關的電影
top_k = 10  # 取前 10 筆
top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]

# 輸出結果
for idx in top_indices:
    print(f"相關電影: {df.iloc[idx]['movie_title']} (分數: {scores[idx]})")


相關電影: 法櫃奇兵 (分數: 2.8361166102161484)
相關電影: 聖戰奇兵 (分數: 2.8361166102161484)
相關電影: 不可能的任務：致命清算 第一章 (分數: 2.8361166102161484)
相關電影: 少林三十六房 (分數: 2.8361166102161484)
相關電影: 不可能的任務：全面瓦解 (分數: 2.8361166102161484)
相關電影: 魔宮傳奇 (分數: 2.8361166102161484)
相關電影: 不可能的任務：失控國度 (分數: 2.8361166102161484)
相關電影: 秘境探險 (分數: 2.8361166102161484)
相關電影: 格羅姆少校：瘟疫醫生 (分數: 2.8361166102161484)
相關電影: 蒙面俠蘇洛 (分數: 2.8361166102161484)


In [89]:
import pandas as pd
import re

# 讀取 Excel 檔案
df = pd.read_excel("dataMovie.xlsx")

# 移除 NaN
df = df.dropna()

# 定義正則表達式，匹配 (xxx) 或 （xxx） 的內容
pattern = r"\((.*?)\)|（(.*?)）"

# 移除 "電影簡述" 欄位中的括號內容
df["movie_description"] = df["movie_description"].apply(lambda x: re.sub(pattern, "", str(x)))

# 存成新的 Excel 檔案
new_file_path = "處理後的電影資料.xlsx"
df.to_excel(new_file_path, index=False)

print(f"處理完成，檔案已儲存為 {new_file_path}")

處理完成，檔案已儲存為 處理後的電影資料.xlsx


In [99]:
# 匯入套件
import jieba
import pandas as pd
import jieba
from rank_bm25 import BM25Okapi

# 加載自定義詞典，避免錯誤切詞
jieba.load_userdict("custom_dict.txt")  # 建立自定義詞典

# 常見中文停用詞列表
stop_words = set([
    '的', '了', '是', '在', '和', '也', '有', '就', '都', '與', '及', '或', '而', '於', '被', '因', '欲', '其',
    '由', '不', '我們', '你們', '他們', '她們', '它們', '自己', '這', '那', '哪', '什麼', '怎麼', '如何', '這次', 
    '為什麼', '呢', '嗎', '吧', '頃刻間', '片中', '即將', '多多', '只能', '如', '哼', '呃', '嗯', '他', '哈哈', '噢', 
    '最', '很', '之間', '不料', '影片', '主要', '講述', '當晚', '本', '嘿喲', '這個', '那個', '這樣', '那樣',
    '某', '每', '各', '所有', '一些', '幾乎', '幾', '仍然', '仍', '既', '又', '還', '但是', '然而', '所以', '除了',
    '因此', '不過', '雖然', '儘管', '如果', '即使', '除非', '或者', '要麼', '不僅', '而且', '此外', '比如',
    '例如', '像', '以及', '甚至', '再者', '總之', '其實', '基本上', '大約', '大概', '或者', '或許', '可能', 
    '應該', '必須', '需要', '得', '能', '可以', '會', '要', '讓', '叫', '成', '給', '向', '跟', '從', 
    '把', '並', '只', '僅', '只是', '才', '即', '乃', '原來', '正是', '簡直', '真是', '特別', '尤其', '倒是', 
    '反而', '反過來', '一種','一位', '一名', '一個', '一株', '一台', '一艘', '無論', '不管', '無法', '無法避免',
    '無法拒絕', '是否', '既然', '當然', '果然', '還是', '或者', '以至於', '因此', '因而', '甚至於', '並且', '不然', 
    '否則', '不如', '不僅如此', '換句話說', '例如', '比如', '拿...來說', '正如', '正所謂', '話說回來', '另一個',
    '進而', '另外', '不過', '還有', '既然', '然而', '總而言之', '總的來說', '總之', '即便', '哪怕', '一場', '一段', 
    '雖然', '儘管', '至於', '則', '反之', '因為', '為了', '由於', '所以', '於是', '假如', '如果', '它', '劇情描述', 
    '要是', '可是', '但', '但是', '然而', '卻', '不過', '只不過', '而已', '除非', '以免', '免得', '竟然', '劇情', 
    '一輛', '一部', '來了之後', '一天', '以其', '一份', '我', '想', '但只要', '當然', '這讓', '本片', '這是', '牠們', '牠', '她', '背景設定', '講述', '電影描述', '故事描述', '劇情敘述', '敘述', '講述', '大家', '各位', '某人', '某些人', '某事', '某種', '這些', '那些', '如此', '這般', '那般', '這時', '那時', '這裡', '那裡', '何時', '何地', '哪裡', '誰', '某個', '某位', '今天', '昨天', '明天', '後天', '大前天', '大後天', '今年', '去年', '明年', '前天', '這時候', '此刻', '剛才', '剛剛', '頃刻', '瞬間', '一會兒', '等會兒', '頃', '不久', '過去', '當年', '長久', '以來', '一點', '一滴', '一絲', '一些', '一堆', '一群', '一批', '一段', '一會兒', '數個', '數十', '數百', '很多', '無數', '千萬', '若干', '一整天', '整個', '大部分', '半數', '少許', '部分', '許多', '幾年', '多年', '數年',  '來', '去', '走', '做', '幹', '進', '出', '到', '看', '聽', '說', '問', '答', '告訴', '回', '開', '關', '起', '坐', '站', '跑', '跳', '拉', '推', '扔', '拿', '給', '帶', '進入', '離開', '變成', '成為', '擁有', '獲得', '得到', '感覺', '覺得', '知道', '了解', '認識', '發現', '明白', '想', '思考', '希望', '簡直', '完全', '十分', '非常', '特別', '極其', '格外', '無比', '超級', '極端', '萬分', '相當', '故事', '影片', '內容', '電影', '描述', '情節', '劇情', '本片', '這部', '這場', '這件', '這部電影', '該片', '本作', '此片', '該電影', '全片', '該作', '這款', '這本', '該故事', '這是一部', '這是一個', '最後', '彼此', '漸漸', '越來越'
])

# 讀取 Excel
df = pd.read_excel("dataMovie.xlsx")

# 移除 NaN
df = df.dropna()

# 確保 "movie_description" 欄位為字串類型
df["movie_description"] = df["movie_description"].astype(str)

# 文本清洗（移除標點符號與數字，但保留空格）
def clean_text(text):
    return ''.join(c if c.isalpha() or c.isspace() else '' for c in text)

df["cleaned_description"] = df["movie_description"].apply(clean_text)

# 斷詞並過濾停用詞
df["tokenized_corpus"] = df["cleaned_description"].apply(
    lambda x: " ".join([word for word in jieba.cut(x) if word not in stop_words])
)

# 顯示前三筆斷詞結果
for i in range(3):
    print(f"第 {i+1} 筆: {df.iloc[i]['tokenized_corpus']}")

# 存成 Excel 方便人工檢查
df[["movie_title", "tokenized_corpus"]].to_excel("tokenized_descriptions.xlsx", index=False)

print("斷詞結果已存成 Excel，請檢查並修改。")

第 1 筆: 銀行家 安迪 本欲 報復 外遇 妻子 行動 之前 打消 主意 其妻 及其 情夫 遭人 殺害 安迪 遂 被判 無期徒刑 鯊堡監獄   獄 中 安迪 事 黑市交易 囚犯 雷並 其他 囚犯 慢慢 熟稔 這同 時 安迪 以 專業知識 逐步 引導 監獄 中 生活 事情 開始 轉變 年 鐵窗生涯 安迪 看透 監獄官僚 貪污腐敗 虛偽 詭計 圈套 體會 還其 清白之身 唯有 逃獄 一 途
第 2 筆: 年代 美國 教父 維托 唐柯 里昂 黑手黨 柯里昂家族 首領 帶領 家族 從事 非法 勾當 同時 弱小平民 保護神 深得 人們 愛戴   拒絕 毒 梟 索洛索 毒品交易 要求 柯里昂家族 紐約 其他 幾個 黑手黨 家族 矛盾激化 聖誕 前夕 索洛索 劫持 教父 參謀 湯姆 並派 人 暗殺 教父 內奸 出賣 教父 大兒子 遜尼 仇家 殺害 小兒子 麥克 捲 進來 失去愛妻 黑手黨 家族 矛盾 越來 越 白熱化   年老 教父 面對 喪子之痛 怎樣 統領全局 黑手黨 仇 殺 落幕 家族 內奸 能夠 新一代 教父   血雨腥風 溫情脈脈 里程碑式 黑幫 史詩 巨片 裡 真實 上演
第 3 筆: 第二代 教父 麥克 柯 里昂 奮鬥歷程 同時 回憶 第一代 教父 維多柯里昂 創業 艱辛 反映 不同 歷史 時期 兩代 教父 事業 家庭生活   麥克 為 兒子 托尼舉 行聖餐 儀式 慶祝活動 當夜 麥克 家中 遭到 襲擊 兇手 被人滅口 面臨 接管 家族事業 重重危機 麥克 回憶 父親 維多柯里昂 年 輕時 美國 創業歷程   麥克 一邊 調查 襲擊 真相 一邊 繼續 開展 賭博 酒店 等 生意 另 黑幫人物 海門羅斯 鬥智斗勇 不斷 擴大勢力   終於 麥克 不法行為 引起 政府 關注 麥克 受到 一系列 指控 同時 麥克 家庭 遇到 危機 夫妻感情 瀕臨破裂 麥克 痛心 家族 中親 人 背叛 第一代 教父 其樂融融 家庭生活 比起 麥克 無疑 失敗   麥克 怎麼樣 面對 事業 家庭 雙重危機 為 兩代 教父 截然不同 家庭生活 中尋找 答案
斷詞結果已存成 Excel，請檢查並修改。


In [48]:
import pandas as pd
import jieba
from rank_bm25 import BM25Okapi

# 讀取 Excel
df = pd.read_excel("dataMovie_clean.xlsx")

# 取得簡述欄位
descriptions = df["movie_description"].dropna().astype(str).tolist()

# 斷詞處理
tokenized_corpus = [list(jieba.cut(desc)) for desc in descriptions]

# 建立 BM25 模型
bm25 = BM25Okapi(tokenized_corpus)

# 用戶輸入
user_query = "荒野機器人 卡通動畫"
tokenized_query = list(jieba.cut(user_query))

# 計算 BM25 分數
scores = bm25.get_scores(tokenized_query)

# 找到最相關的電影
top_k = 3  # 取前 3 筆
top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]

# 輸出結果
for idx in top_indices:
    print(f"相關電影: {df.iloc[idx]['movie_title']} (分數: {scores[idx]})")


相關電影: 未來世代 (分數: 15.86202661262035)
相關電影: 米奇耶誕嘉年華 (分數: 14.99197014198878)
相關電影: 風中奇緣2：倫敦之旅 (分數: 14.801463203898098)


In [101]:
import jieba

# 加載自定義詞典，確保專有名詞不會被錯誤切開
jieba.load_userdict("custom_dict.txt")  

# 停用詞
stopwords = set([
    '的', '了', '是', '在', '和', '也', '有', '就', '都', '與', '及', '或', '而', '於', '被', '因', '欲', '其',
    '由', '不', '我們', '你們', '他們', '她們', '它們', '自己', '這', '那', '哪', '什麼', '怎麼', '如何', '這次', 
    '為什麼', '呢', '嗎', '吧', '頃刻間', '片中', '即將', '多多', '只能', '如', '哼', '呃', '嗯', '他', '哈哈', '噢', 
    '最', '很', '之間', '不料', '影片', '主要', '講述', '當晚', '本', '嘿喲', '這個', '那個', '這樣', '那樣',
    '某', '每', '各', '所有', '一些', '幾乎', '幾', '仍然', '仍', '既', '又', '還', '但是', '然而', '所以', '除了',
    '因此', '不過', '雖然', '儘管', '如果', '即使', '除非', '或者', '要麼', '不僅', '而且', '此外', '比如',
    '例如', '像', '以及', '甚至', '再者', '總之', '其實', '基本上', '大約', '大概', '或者', '或許', '可能', 
    '應該', '必須', '需要', '得', '能', '可以', '會', '要', '讓', '叫', '成', '給', '向', '跟', '從', 
    '把', '並', '只', '僅', '只是', '才', '即', '乃', '原來', '正是', '簡直', '真是', '特別', '尤其', '倒是', 
    '反而', '反過來', '一種','一位', '一名', '一個', '一株', '一台', '一艘', '無論', '不管', '無法', '無法避免',
    '無法拒絕', '是否', '既然', '當然', '果然', '還是', '或者', '以至於', '因此', '因而', '甚至於', '並且', '不然', 
    '否則', '不如', '不僅如此', '換句話說', '例如', '比如', '拿...來說', '正如', '正所謂', '話說回來', '另一個',
    '進而', '另外', '不過', '還有', '既然', '然而', '總而言之', '總的來說', '總之', '即便', '哪怕', '一場', '一段', 
    '雖然', '儘管', '至於', '則', '反之', '因為', '為了', '由於', '所以', '於是', '假如', '如果', '它', '劇情描述', 
    '要是', '可是', '但', '但是', '然而', '卻', '不過', '只不過', '而已', '除非', '以免', '免得', '竟然', '劇情', 
    '一輛', '一部', '來了之後', '一天', '以其', '一份', '我', '想', '但只要', '當然', '這讓', '本片', '這是', '牠們', '牠', '她', '背景設定', '講述', '電影描述', '故事描述', '劇情敘述', '敘述', '講述', '大家', '各位', '某人', '某些人', '某事', '某種', '這些', '那些', '如此', '這般', '那般', '這時', '那時', '這裡', '那裡', '何時', '何地', '哪裡', '誰', '某個', '某位', '今天', '昨天', '明天', '後天', '大前天', '大後天', '今年', '去年', '明年', '前天', '這時候', '此刻', '剛才', '剛剛', '頃刻', '瞬間', '一會兒', '等會兒', '頃', '不久', '過去', '當年', '長久', '以來', '一點', '一滴', '一絲', '一些', '一堆', '一群', '一批', '一段', '一會兒', '數個', '數十', '數百', '很多', '無數', '千萬', '若干', '一整天', '整個', '大部分', '半數', '少許', '部分', '許多', '幾年', '多年', '數年',  '來', '去', '走', '做', '幹', '進', '出', '到', '看', '聽', '說', '問', '答', '告訴', '回', '開', '關', '起', '坐', '站', '跑', '跳', '拉', '推', '扔', '拿', '給', '帶', '進入', '離開', '變成', '成為', '擁有', '獲得', '得到', '感覺', '覺得', '知道', '了解', '認識', '發現', '明白', '想', '思考', '希望', '簡直', '完全', '十分', '非常', '特別', '極其', '格外', '無比', '超級', '極端', '萬分', '相當', '故事', '影片', '內容', '電影', '描述', '情節', '劇情', '本片', '這部', '這場', '這件', '這部電影', '該片', '本作', '此片', '該電影', '全片', '該作', '這款', '這本', '該故事', '這是一部', '這是一個', '最後', '彼此', '漸漸', '越來越'
])

# 原始文本
text = "賴納•文格爾 是德國某所高中的老師，該學校正在進行“國家體制”的主題活動周。由於他最喜歡的“無政府主義”課被另一位老師捷足先登，因此他只能主講“獨裁統治”課程。 對於自由散漫的學生們來說，任何課程都只是為了學分而上。他們在課上大聲聊天，無心聽講。文格爾別出心裁提出假想“獨裁”的實驗。在為期一周的實驗中，文格爾被置於至高無上的地位，學生們對他要絕對服從。從最初的玩樂心態，這些青年男女漸漸沉湎這個名為“浪潮”的組織中，他們體會到集體和紀律的重要性，卻在不知不覺中滑向了“獨裁”與“納粹”的深淵……"

# 斷詞並去停用詞
seg_list = jieba.cut(text, cut_all=False)
filtered_words = [word for word in seg_list if word not in stopwords]

# 顯示結果
print(" | ".join(filtered_words))

賴納 | • | 文格爾 |   | 德國 | 所 | 高中 | 老師 | ， | 該學 | 校正 | 進行 | “ | 國家 | 體制 | ” | 主題 | 活動周 | 。 | 喜歡 | “ | 無 | 政府 | 主義 | ” | 課 | 另 | 老師 | 捷足先登 | ， | 主講 | “ | 獨裁 | 統治 | ” | 課程 | 。 |   | 對 | 自由散漫 | 學生 | 們 | ， | 任何 | 課程 | 為 | 學分 | 上 | 。 | 課上 | 大聲 | 聊天 | ， | 無心 | 聽講 | 。 | 文格爾別 | 心裁 | 提出 | 假想 | “ | 獨裁 | ” | 實驗 | 。 | 為期 | 一周 | 實驗 | 中 | ， | 文格爾 | 置 | 至 | 高無上 | 地位 | ， | 學生 | 們 | 對 | 絕對服 | 。 | 最初 | 玩樂 | 心態 | ， | 青年 | 男女 | 沉湎 | 名為 | “ | 浪潮 | ” | 組織 | 中 | ， | 體會 | 集體 | 紀律 | 重要性 | ， | 不知 | 不覺 | 中 | 滑向 | “ | 獨裁 | ” | “ | 納粹 | ” | 深淵 | … | …


In [46]:
import numpy as np
import pickle
import faiss
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 設定模型名稱
MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

# 檔案路徑
MOVIES_INDEX_PATH = './movies_excel_vector.index'
GAMES_INDEX_PATH = './games_excel_vector.index'
MOVIES_IDS_PATH = './movies_excel_ids.pkl'
GAMES_IDS_PATH = './games_excel_ids.pkl'

# 載入模型
print(f"載入 {MODEL_NAME} 模型...")
model = SentenceTransformer(MODEL_NAME)

# 載入事先計算好的FAISS索引
print("載入電影與遊戲FAISS索引...")
movie_vectors = faiss.read_index(MOVIES_INDEX_PATH)
game_vectors = faiss.read_index(GAMES_INDEX_PATH)

print(f"Movie vectors: {movie_vectors.ntotal} 條向量, 維度: {movie_vectors.d}")
print(f"Game vectors: {game_vectors.ntotal} 條向量, 維度: {game_vectors.d}")


# 載入 ID 映射
print("載入電影與遊戲 ID 映射...")
with open(MOVIES_IDS_PATH, 'rb') as f:
    movie_ids = pickle.load(f)
with open(GAMES_IDS_PATH, 'rb') as f:
    game_ids = pickle.load(f)

def search_by_semantic(user_query, is_movie=True, top_k=5):
    """使用語義搜尋電影或遊戲"""
    print(f"搜尋語義相似度... (類型: {'電影' if is_movie else '遊戲'})")
    
    # 取得查詢向量
    query_vector = model.encode([user_query], normalize_embeddings=True)
    
    # 選擇對應的向量庫
    vectors = movie_vectors if is_movie else game_vectors
    
    # 使用 FAISS 進行相似度搜尋
    top_k = min(top_k, vectors.ntotal)  # 避免超出索引範圍
    D, I = vectors.search(query_vector, top_k)

    # 根據 ID 映射回傳對應的標題與相似度分數
    results = [(movie_ids[i] if is_movie else game_ids[i], D[0][idx]) for idx, i in enumerate(I[0]) if i != -1]
    
    return results

# 測試搜尋
query = "宇宙 星際大戰"
result_movies = search_by_semantic(query, is_movie=True)
result_games = search_by_semantic(query, is_movie=False)

print("\n🎬 最相似的電影:")
for title, score in result_movies:
    print(f"{title}: {score:.4f}")

print("\n🎮 最相似的遊戲:")
for title, score in result_games:
    print(f"{title}: {score:.4f}")

載入 sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 模型...


c:\Users\User\anaconda3\envs\web_scraping\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


載入電影與遊戲FAISS索引...
Movie vectors: 7234 條向量, 維度: 384
Game vectors: 13197 條向量, 維度: 384
載入電影與遊戲 ID 映射...
搜尋語義相似度... (類型: 電影)
搜尋語義相似度... (類型: 遊戲)

🎬 最相似的電影:
{'title': '星際大戰二部曲：複製人全面進攻', 'genre': '冒險, 動作, 科幻', 'type': 'movie'}: 0.7204
{'title': '復仇者聯盟：無限之戰', 'genre': '冒險, 動作, 科幻', 'type': 'movie'}: 0.7178
{'title': '環太平洋2：起義時刻', 'genre': '動作, 奇幻, 科幻, 冒險', 'type': 'movie'}: 0.7093
{'title': '星際大戰外傳：韓索羅', 'genre': '科幻, 冒險, 動作', 'type': 'movie'}: 0.6817
{'title': '明日邊界', 'genre': '動作, 科幻', 'type': 'movie'}: 0.6779

🎮 最相似的遊戲:
{'title': '銀河司令部', 'genre': '飛行射擊', 'type': 'game'}: 0.7294
{'title': '索爾之戰', 'genre': '飛行射擊', 'type': 'game'}: 0.7227
{'title': '宇宙戰爭', 'genre': '模擬遊戲', 'type': 'game'}: 0.6921
{'title': '深入敵後：雷神戰爭', 'genre': '第一人稱射擊', 'type': 'game'}: 0.6871
{'title': '礦工戰場', 'genre': '飛行射擊', 'type': 'game'}: 0.6775
